## `Part 1: Generate dataset`

In [ ]:
import cv2
import os

def generate_dataset():
    # Ensure the cascade file exists
    cascade_path = "haarcascade_frontalface_default.xml"
    if not os.path.exists(cascade_path):
        print(f"Error: Cascade file '{cascade_path}' not found.")
        return
    
    face_classifier = cv2.CascadeClassifier(cascade_path)
    
    # Ensure data directory exists
    if not os.path.exists("data"):
        os.makedirs("data")
    
    def face_cropped(img):
        if img is None:
            return None
        
        try:
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            faces = face_classifier.detectMultiScale(gray, 1.3, 5)
            
            if len(faces) == 0:  # Fixed: use len() instead of comparing with ()
                return None
                
            for (x, y, w, h) in faces:
                cropped_face = img[y:y+h, x:x+w]
                return cropped_face
                
            return None
        except Exception as e:
            print(f"Error in face_cropped: {e}")
            return None
    
    # Try different camera indices
    camera_index = 0
    max_attempts = 3
    
    for i in range(max_attempts):
        print(f"Trying camera index {camera_index}")
        cap = cv2.VideoCapture(camera_index, cv2.CAP_DSHOW)  # Use DirectShow on Windows
        
        if cap.isOpened():
            print(f"Successfully opened camera with index {camera_index}")
            # Wait for camera to initialize
            import time
            time.sleep(2)
            ret, test_frame = cap.read()
            if ret and test_frame is not None:
                print("Camera providing valid frames")
                break
        
        print(f"Failed with camera index {camera_index}")
        cap.release()
        camera_index += 1
        
        if i == max_attempts - 1:
            print("Could not initialize any camera")
            return
    
    id = 1
    img_id = 0
    
    while True:
        ret, frame = cap.read()
        
        if not ret or frame is None:
            print("Failed to capture frame")
            break
            
        face = face_cropped(frame)
        
        if face is not None:
            img_id += 1
            face = cv2.resize(face, (200, 200))
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
            file_name_path = f"data/user.{id}.{img_id}.jpg"
            cv2.imwrite(file_name_path, face)
            cv2.putText(face, str(img_id), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)
            
            cv2.imshow("Cropped face", face)
            
            # Show the original frame with rectangle
            cv2.rectangle(frame, (frame.shape[1]//2-100, frame.shape[0]//2-100), 
                         (frame.shape[1]//2+100, frame.shape[0]//2+100), (0, 255, 0), 2)
            cv2.imshow("Video Frame", frame)
        
        if cv2.waitKey(1) == 13 or img_id >= 200:  # 13 is the ASCII character of Enter
            break
    
    cap.release()
    cv2.destroyAllWindows()
    print(f"Collecting samples completed. Collected {img_id} samples.")

generate_dataset()

## `Part 2: Train the classifier and save it`

In [ ]:
from PIL import Image
import numpy as np
import os
import cv2

def train_classifier(data_dir):
    # Check if data directory exists
    if not os.path.exists(data_dir):
        print(f"Error: Data directory '{data_dir}' not found.")
        return
    
    # Get list of image files
    path = [os.path.join(data_dir, f) for f in os.listdir(data_dir)]
    if not path:
        print(f"Error: No files found in '{data_dir}'.")
        return
    
    faces = []
    ids = []
    
    for image in path:
        try:
            img = Image.open(image).convert('L')
            imageNp = np.array(img, 'uint8')
            id = int(os.path.split(image)[1].split(".")[1])
            
            faces.append(imageNp)
            ids.append(id)
            # print(f"Processing image: {os.path.basename(image)}")
        except Exception as e:
            print(f"Error processing {image}: {e}")
    
    if not faces:
        print("No valid face images found.")
        return
        
    ids = np.array(ids)
    
    # Train and save classifier
    clf = cv2.face.LBPHFaceRecognizer_create()
    print(f"Training with {len(faces)} images...")
    clf.train(faces, ids)
    clf.write("classifier.xml")
    print("Training completed. Classifier saved as 'classifier.xml'")

train_classifier("data")

## `Part 3: Face detection/recognition`

In [ ]:
import cv2
import numpy as np
from PIL import Image
import os
 
def draw_boundary(img, classifier, scaleFactor, minNeighbors, color, text, clf):
    if img is None:
        return img
        
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    features = classifier.detectMultiScale(gray_img, scaleFactor, minNeighbors)
     
    for (x,y,w,h) in features:
        cv2.rectangle(img, (x,y), (x+w,y+h), color, 2 )
         
        id, pred = clf.predict(gray_img[y:y+h,x:x+w])
        confidence = int(100*(1-pred/300))
         
        if confidence>85:
            if id==1:
                cv2.putText(img, "Sujal", (x,y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)
            if id==2:
                cv2.putText(img, "Manish", (x,y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)
        else:
            cv2.putText(img, "UNKNOWN", (x,y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,255), 1, cv2.LINE_AA)
     
    return img
 
# Check for required files
if not os.path.exists("haarcascade_frontalface_default.xml"):
    print("Error: Missing face cascade file")
    exit()
if not os.path.exists("classifier.xml"):
    print("Error: Missing classifier model file")
    exit()

# Loading classifier
faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
 
clf = cv2.face.LBPHFaceRecognizer_create()
clf.read("classifier.xml")
 
# Try to open camera
video_capture = cv2.VideoCapture(0, cv2.CAP_DSHOW)  # Use DirectShow on Windows

if not video_capture.isOpened():
    print("Error: Could not open camera")
    exit()

window_name = "Face Detection"
cv2.namedWindow(window_name)
 
while True:
    # Check if window is closed
    if cv2.getWindowProperty(window_name, cv2.WND_PROP_VISIBLE) < 1:
        print("Window closed by user")
        break
        
    ret, img = video_capture.read()
    if not ret or img is None:
        print("Error reading frame")
        break
        
    img = draw_boundary(img, faceCascade, 1.3, 6, (255,255,255), "Face", clf)
    cv2.imshow(window_name, img)

    key = cv2.waitKey(1)
    if key == 13 or key == ord('q'):  # 13 is the Enter key
        print("Exiting...")
        break

video_capture.release()
cv2.destroyAllWindows()

## `Part 4 Converting the above project into GUI`

In [12]:
import tkinter as tk
import os
import cv2
import numpy as np
from PIL import Image
from tkinter import messagebox
 
window = tk.Tk()
window.title("Face Recognition system")
 
l1 = tk.Label(window, text="Name", font=("Algerian",20))
l1.grid(column=0, row=0)
t1 = tk.Entry(window, width=50, bd=5)
t1.grid(column=1, row=0)
 
l2 = tk.Label(window, text="Age", font=("Algerian",20))
l2.grid(column=0, row=1)
t2 = tk.Entry(window, width=50, bd=5)
t2.grid(column=1, row=1)
 
l3 = tk.Label(window, text="Address", font=("Algerian",20))
l3.grid(column=0, row=2)
t3 = tk.Entry(window, width=50, bd=5)
t3.grid(column=1, row=2)

def train_classifier():
    data_dir = "C:/Users/sujal/OneDrive/Desktop/new2/data"
    # Check if data directory exists
    if not os.path.exists(data_dir):
        print(f"Error: Data directory '{data_dir}' not found.")
        return
    
    # Get list of image files
    path = [os.path.join(data_dir, f) for f in os.listdir(data_dir)]
    if not path:
        print(f"Error: No files found in '{data_dir}'.")
        return
    
    faces = []
    ids = []
    
    for image in path:
        try:
            img = Image.open(image).convert('L')
            imageNp = np.array(img, 'uint8')
            id = int(os.path.split(image)[1].split(".")[1])
            
            faces.append(imageNp)
            ids.append(id)
            # print(f"Processing image: {os.path.basename(image)}")
        except Exception as e:
            print(f"Error processing {image}: {e}")
    
    if not faces:
        print("No valid face images found.")
        return
        
    ids = np.array(ids)
    
    # Train and save classifier
    clf = cv2.face.LBPHFaceRecognizer_create()
    print(f"Training with {len(faces)} images...")
    clf.train(faces, ids)
    clf.write("classifier.xml")
    messagebox.showinfo("Result", "Training completed. Classifier saved as 'classifier.xml'")
    print("Training completed. Classifier saved as 'classifier.xml'")

b1 = tk.Button(window, text="Training", font=("Algerian",20),bg="orange",fg="red", command=train_classifier)
# b1.config(state=tk.DISABLED)  # Disable the button initially
b1.grid(column=0, row=4)

def detect_faces():
    def draw_boundary(img, classifier, scaleFactor, minNeighbors, color, text, clf):
        if img is None:
            return img
            
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        features = classifier.detectMultiScale(gray_img, scaleFactor, minNeighbors)
        
        for (x,y,w,h) in features:
            cv2.rectangle(img, (x,y), (x+w,y+h), color, 2 )
            
            id, pred = clf.predict(gray_img[y:y+h,x:x+w])
            confidence = int(100*(1-pred/300))
            
            if confidence>85:
                if id==1:
                    cv2.putText(img, "Sujal", (x,y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)
                if id==2:
                    cv2.putText(img, "Manish", (x,y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)
            else:
                cv2.putText(img, "UNKNOWN", (x,y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,255), 1, cv2.LINE_AA)
        
        return img
    
    # Check for required files
    if not os.path.exists("haarcascade_frontalface_default.xml"):
        print("Error: Missing face cascade file")
        exit()
    if not os.path.exists("classifier.xml"):
        print("Error: Missing classifier model file")
        exit()

    # Loading classifier
    faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    
    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.read("classifier.xml")
    
    # Try to open camera
    video_capture = cv2.VideoCapture(0, cv2.CAP_DSHOW)  # Use DirectShow on Windows

    if not video_capture.isOpened():
        print("Error: Could not open camera")
        exit()

    window_name = "Face Detection"
    cv2.namedWindow(window_name)
    
    while True:
        # Check if window is closed
        if cv2.getWindowProperty(window_name, cv2.WND_PROP_VISIBLE) < 1:
            print("Window closed by user")
            break
            
        ret, img = video_capture.read()
        if not ret or img is None:
            print("Error reading frame")
            break
            
        img = draw_boundary(img, faceCascade, 1.3, 6, (255,255,255), "Face", clf)
        cv2.imshow(window_name, img)

        key = cv2.waitKey(1)
        if key == 13 or key == ord('q'):  # 13 is the Enter key
            print("Exiting...")
            break

    video_capture.release()
    cv2.destroyAllWindows()

b2 = tk.Button(window, text="Detect the faces", font=("Algerian",20), bg="green", fg="orange", command=detect_faces)
# b2.config(state=tk.DISABLED)  # Disable the button initially
b2.grid(column=1, row=4)
 
def generate_dataset():
    if(t1.get()=="" or t2.get()=="" or t3.get()==""):
        messagebox.showinfo('Result','Please provide complete details of the user')

    else:
      # Ensure the cascade file exists
      cascade_path = "haarcascade_frontalface_default.xml"
      if not os.path.exists(cascade_path):
          print(f"Error: Cascade file '{cascade_path}' not found.")
          return
      
      face_classifier = cv2.CascadeClassifier(cascade_path)
      
      # Ensure data directory exists
      if not os.path.exists("data"):
          os.makedirs("data")
      
      def face_cropped(img):
          if img is None:
              return None
          
          try:
              gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
              faces = face_classifier.detectMultiScale(gray, 1.3, 5)
              
              if len(faces) == 0:  # Fixed: use len() instead of comparing with ()
                  return None
                  
              for (x, y, w, h) in faces:
                  cropped_face = img[y:y+h, x:x+w]
                  return cropped_face
                  
              return None
          except Exception as e:
              print(f"Error in face_cropped: {e}")
              return None
      
      # Try different camera indices
      camera_index = 0
      max_attempts = 3
      
      for i in range(max_attempts):
          print(f"Trying camera index {camera_index}")
          cap = cv2.VideoCapture(camera_index, cv2.CAP_DSHOW)  # Use DirectShow on Windows
          
          if cap.isOpened():
              print(f"Successfully opened camera with index {camera_index}")
              # Wait for camera to initialize
              import time
              time.sleep(2)
              ret, test_frame = cap.read()
              if ret and test_frame is not None:
                  print("Camera providing valid frames")
                  break
          
          print(f"Failed with camera index {camera_index}")
          cap.release()
          camera_index += 1
          
          if i == max_attempts - 1:
              print("Could not initialize any camera")
              return
      
      id = 1
      img_id = 0
      
      while True:
          ret, frame = cap.read()
          
          if not ret or frame is None:
              print("Failed to capture frame")
              break
              
          face = face_cropped(frame)
          
          if face is not None:
              img_id += 1
              face = cv2.resize(face, (200, 200))
              face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
              file_name_path = f"data/user.{id}.{img_id}.jpg"
              cv2.imwrite(file_name_path, face)
              cv2.putText(face, str(img_id), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)
              
              cv2.imshow("Cropped face", face)
              
              # Show the original frame with rectangle
              cv2.rectangle(frame, (frame.shape[1]//2-100, frame.shape[0]//2-100), 
                          (frame.shape[1]//2+100, frame.shape[0]//2+100), (0, 255, 0), 2)
              cv2.imshow("Video Frame", frame)
          
          if cv2.waitKey(1) == 13 or img_id >= 200:  # 13 is the ASCII character of Enter
              break
      
      cap.release()
      cv2.destroyAllWindows()
      messagebox.showinfo("Result", "Generating dataset completed")

b3 = tk.Button(window, text="Generate dataset", font=("Algerian",20), bg="pink", fg="black", command=generate_dataset)
b3.grid(column=2, row=4)
 
window.geometry("800x200")
window.mainloop()